# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from keras.preprocessing.image import ImageDataGenerator as data_augment
from keras.models import Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Data Importation


In [2]:
#Uncomment below to clone the dataset from github

!git clone https://github.com/echefulouis/Gender-Classification-Model.git


Cloning into 'Gender-Classification-Model'...
remote: Enumerating objects: 58653, done.
remote: Total 58653 (delta 0), reused 0 (delta 0), pack-reused 58653
Receiving objects: 100% (58653/58653), 261.43 MiB | 16.00 MiB/s, done.
Updating files: 100% (58659/58659), done.


# Data Preprocessing and Augmentation

In [3]:
data_generate_training = data_augment (rescale=1./255,  #This parameter rescales the pixel values of the image to a value between 0 and 1
                              shear_range = 0.2,        #This parameter randomly applies a shear transformation to the image, which skews the image 
                                                        #in a particular direction.
                              zoom_range = 0.2,
                              fill_mode = "nearest",
                              horizontal_flip = True,
                              width_shift_range = 0.2,
                              height_shift_range = 0.2,
                              
                              validation_split = 0.15)  #This parameter splits the training data into a training set and a validation set, 
                                                        #with 15% of the data being used for validation.

data_generate_test = data_augment(rescale = 1./255)     #This parameter rescales the pixel values of the image to a value between 0 and 1

In [4]:
train_data = data_generate_training.flow_from_directory("/content/Gender-Classification-Model/GC dataset/Training",
                                          target_size = (96, 96),
                                          seed = 123,
                                          batch_size = 32,
                                          subset = "training")

Found 39959 images belonging to 2 classes.


In [5]:


test_data = data_generate_training.flow_from_directory("/content/Gender-Classification-Model/GC dataset/Validation",
                                          target_size = (96, 96),
                                          seed = 123,
                                          batch_size = 32,
                                          subset = "validation")

Found 1747 images belonging to 2 classes.


# Model Creation and Training 

In [6]:
CNNmodel = keras.Sequential([
    layers.Conv2D(32, (3, 3), input_shape=(96, 96, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.2),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.2),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.2),
    
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.2),  
    layers.Dense(128, activation = 'relu'),
    layers.Dense(2, activation = 'sigmoid')
])


In [7]:

CNNmodel.compile(optimizer='adam',
              loss="binary_crossentropy",
              metrics=['accuracy'])

history = CNNmodel.fit(train_data, epochs = 10, validation_data = test_data) 

Epoch 1/10
1249/1249 [==============================] - 151s 111ms/step - loss: 0.4411 - accuracy: 0.8138 - val_loss: 0.3650 - val_accuracy: 0.8357
Epoch 2/10
1249/1249 [==============================] - 139s 111ms/step - loss: 0.2278 - accuracy: 0.9104 - val_loss: 0.3570 - val_accuracy: 0.8357
Epoch 3/10
1249/1249 [==============================] - 139s 111ms/step - loss: 0.1933 - accuracy: 0.9273 - val_loss: 0.3054 - val_accuracy: 0.8809
Epoch 4/10
1249/1249 [==============================] - 136s 109ms/step - loss: 0.1751 - accuracy: 0.9352 - val_loss: 0.1481 - val_accuracy: 0.9450
Epoch 5/10
1249/1249 [==============================] - 134s 107ms/step - loss: 0.1633 - accuracy: 0.9397 - val_loss: 0.2138 - val_accuracy: 0.9296
Epoch 6/10
1249/1249 [==============================] - 132s 106ms/step - loss: 0.1551 - accuracy: 0.9443 - val_loss: 0.1639 - val_accuracy: 0.9359
Epoch 7/10
1249/1249 [==============================] - 134s 107ms/step - loss: 0.1485 - accuracy: 0.9455 - val_

In [8]:
# Evaluate the model on the validation dataset
validation_accuracy = CNNmodel.evaluate(test_data)[1]

print("Validation accuracy: {:.2f}%".format(validation_accuracy * 100))

55/55 [==============================] - 5s 87ms/step - loss: 0.1352 - accuracy: 0.9536
Validation accuracy: 95.36%


#Transfer Learning 